## Imports

In [51]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [52]:
import numpy as np
import sys
LIBS_AI = '/home/ubuntu/fastai/'
sys.path.append(LIBS_AI)

In [53]:
from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *

## Load Data

In [54]:
DATA_PATH_EN = '../data/comtrans_en.txt'
DATA_PATH_DE = '../data/comtrans_de.txt'

In [55]:
raw_data = open(DATA_PATH_EN).read()

In [56]:
print('corpus length:', len(raw_data))

corpus length: 3885310


In [57]:
raw_data[:400]

"Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Friday 17 December 1999 , and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .\nAlthough , as you will have seen , the dreaded ' millennium bug ' failed to materialise , still the people in a number of countries suffered a series of natural dis"

## Preprocessing

In [58]:
chars = sorted(list(set(raw_data)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 83


In [59]:
''.join(chars)

'\n !"$%&\'()+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz'

In [60]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

In [61]:
idx = [char_indices[c] for c in raw_data]
idx[:15]

[45, 60, 74, 76, 68, 71, 75, 64, 70, 69, 1, 70, 61, 1, 75]

In [62]:
''.join(indices_char[i] for i in idx[:15])

'Resumption of t'

In [13]:
cs=8

In [63]:
c_in_dat = [[idx[i+j] for i in range(cs)] for j in range(len(idx)-cs)]
c_out_dat = [idx[j+cs] for j in range(len(idx)-cs)]

In [65]:
c_in_dat

[[45, 60, 74, 76, 68, 71, 75, 64],
 [60, 74, 76, 68, 71, 75, 64, 70],
 [74, 76, 68, 71, 75, 64, 70, 69],
 [76, 68, 71, 75, 64, 70, 69, 1],
 [68, 71, 75, 64, 70, 69, 1, 70],
 [71, 75, 64, 70, 69, 1, 70, 61],
 [75, 64, 70, 69, 1, 70, 61, 1],
 [64, 70, 69, 1, 70, 61, 1, 75],
 [70, 69, 1, 70, 61, 1, 75, 63],
 [69, 1, 70, 61, 1, 75, 63, 60],
 [1, 70, 61, 1, 75, 63, 60, 1],
 [70, 61, 1, 75, 63, 60, 1, 74],
 [61, 1, 75, 63, 60, 1, 74, 60],
 [1, 75, 63, 60, 1, 74, 60, 74],
 [75, 63, 60, 1, 74, 60, 74, 74],
 [63, 60, 1, 74, 60, 74, 74, 64],
 [60, 1, 74, 60, 74, 74, 64, 70],
 [1, 74, 60, 74, 74, 64, 70, 69],
 [74, 60, 74, 74, 64, 70, 69, 0],
 [60, 74, 74, 64, 70, 69, 0, 36],
 [74, 74, 64, 70, 69, 0, 36, 1],
 [74, 64, 70, 69, 0, 36, 1, 59],
 [64, 70, 69, 0, 36, 1, 59, 60],
 [70, 69, 0, 36, 1, 59, 60, 58],
 [69, 0, 36, 1, 59, 60, 58, 67],
 [0, 36, 1, 59, 60, 58, 67, 56],
 [36, 1, 59, 60, 58, 67, 56, 73],
 [1, 59, 60, 58, 67, 56, 73, 60],
 [59, 60, 58, 67, 56, 73, 60, 1],
 [60, 58, 67, 56, 73, 60, 

In [64]:
c_out_dat

[70,
 69,
 1,
 70,
 61,
 1,
 75,
 63,
 60,
 1,
 74,
 60,
 74,
 74,
 64,
 70,
 69,
 0,
 36,
 1,
 59,
 60,
 58,
 67,
 56,
 73,
 60,
 1,
 73,
 60,
 74,
 76,
 68,
 60,
 59,
 1,
 75,
 63,
 60,
 1,
 74,
 60,
 74,
 74,
 64,
 70,
 69,
 1,
 70,
 61,
 1,
 75,
 63,
 60,
 1,
 32,
 76,
 73,
 70,
 71,
 60,
 56,
 69,
 1,
 43,
 56,
 73,
 67,
 64,
 56,
 68,
 60,
 69,
 75,
 1,
 56,
 59,
 65,
 70,
 76,
 73,
 69,
 60,
 59,
 1,
 70,
 69,
 1,
 33,
 73,
 64,
 59,
 56,
 80,
 1,
 16,
 22,
 1,
 31,
 60,
 58,
 60,
 68,
 57,
 60,
 73,
 1,
 16,
 24,
 24,
 24,
 1,
 11,
 1,
 56,
 69,
 59,
 1,
 36,
 1,
 78,
 70,
 76,
 67,
 59,
 1,
 67,
 64,
 66,
 60,
 1,
 70,
 69,
 58,
 60,
 1,
 56,
 62,
 56,
 64,
 69,
 1,
 75,
 70,
 1,
 78,
 64,
 74,
 63,
 1,
 80,
 70,
 76,
 1,
 56,
 1,
 63,
 56,
 71,
 71,
 80,
 1,
 69,
 60,
 78,
 1,
 80,
 60,
 56,
 73,
 1,
 64,
 69,
 1,
 75,
 63,
 60,
 1,
 63,
 70,
 71,
 60,
 1,
 75,
 63,
 56,
 75,
 1,
 80,
 70,
 76,
 1,
 60,
 69,
 65,
 70,
 80,
 60,
 59,
 1,
 56,
 1,
 71,
 67,
 60,
 56,
 74,
 56,


In [66]:
xs = np.stack(c_in_dat, axis=0)

In [67]:
xs.shape

(3885302, 8)

In [68]:
y = np.stack(c_out_dat)

In [69]:
val_idx = get_cv_idxs(len(idx)-cs-1)

## RNN Model

In [19]:
n_fac = 30; n_hidden = 10;

In [71]:
class CharRnn(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(1, bs, n_hidden))
        inp = self.e(torch.stack(cs))
        outp,h = self.rnn(inp, h)
        
        return F.log_softmax(self.l_out(outp[-1]), dim=-1)

In [72]:
md = ColumnarModelData.from_arrays('.', val_idx, xs, y, bs=512)

In [73]:
m = CharRnn(vocab_size, n_fac).cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [74]:
fit(m, md, 2, opt, F.nll_loss)

epoch      trn_loss   val_loss                                
    0      2.201607   2.194025  
    1      2.148035   2.1581                                  



[array([2.1581])]

In [29]:
def get_next(inp):
    idxs = T(np.array([char_indices[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return chars[i]

In [36]:
get_next('make america great agai')

'n'

In [37]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [42]:
get_next_n('make america great a', n=5)

'make america great and th'

## Tuning RNN 

In [43]:
set_lrs(opt, 1e-2)
fit(m, md, 5, opt, F.nll_loss)

epoch      trn_loss   val_loss                                
    0      2.108399   2.111457  
    1      2.11694    2.114477                                
    2      2.129302   2.120843                                
    3      2.106824   2.109729                                
    4      2.120976   2.119437                                



[array([2.11944])]

In [44]:
get_next_n('teach', n=5)

'teach the '